In [1]:
import WebNLG_xmlReader.benchmark_reader as xml_reader
import os.path
import pickle
import pandas as pd
import numpy as np
import re
import torch 
import tensorflow as tf
from transformers import AutoTokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline

from datasets import load_dataset


e:\Anaconda\envs\pytorch_test\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#import model
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

e:\Anaconda\envs\pytorch_test\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


## Data Preperation

In [3]:
full_dataset = load_dataset("../Data/Cleaned_data/", data_files={"train": "Train.csv", "dev": "Dev.csv", "test": "Test.csv"})

Using custom data configuration Cleaned_data-bcb014efcf526ad6
Found cached dataset csv (C:/Users/Simon/.cache/huggingface/datasets/csv/Cleaned_data-bcb014efcf526ad6/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
100%|██████████| 3/3 [00:00<00:00, 749.43it/s]


In [4]:
# zie https://github.com/NielsRogge/Transformers-Tutorials/blob/master/T5/Fine_tuning_Dutch_T5_base_on_CNN_Daily_Mail_for_summarization_(on_TPU_using_HuggingFace_Accelerate).ipynb

def preprocess_data(data):
    max_length = 256
    RDFs = data["input"]
    texts = data["output"]

    model_inputs = tokenizer(RDFs, truncation=True, padding="max_length", return_tensors='pt')

    # specially for seq2seq tokenizer, "Temporarily sets the tokenizer for encoding the targets. Useful for tokenizer associated to sequence-to-sequence models that need a slightly different processing for the labels."
    with tokenizer.as_target_tokenizer():
        target_texts = tokenizer(texts, padding="max_length", truncation=True, return_tensors='pt').input_ids
    
    model_inputs["labels"] = target_texts
    
    return model_inputs

In [5]:
encoded_train_ds = full_dataset["train"].map(preprocess_data, batched=True, remove_columns=full_dataset["train"].column_names)  
encoded_dev_ds = full_dataset["dev"].map(preprocess_data, batched=True, remove_columns=full_dataset["dev"].column_names)
encoded_test_ds = full_dataset["test"].map(preprocess_data, batched=True, remove_columns=full_dataset["test"].column_names)

encoded_train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_dev_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# create smaller dataset chunk
small_train = encoded_train_ds.shard(num_shards = 8, index = 0)
small_val = encoded_dev_ds.shard(num_shards = 8, index = 0)
small_test = encoded_test_ds.shard(num_shards = 8, index = 0)

Parameter 'function'=<function preprocess_data at 0x0000021E0E2351F8> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 10/10 [00:01<00:00,  7.37ba/s]


## Model Prep

In [6]:
# pip install datasets
import datasets
bleu = datasets.load_metric("bleu")
def postprocess_text(preds, labels):
    preds = [pred.split() for pred in preds]
    labels = [[label.split()] for label in labels]
    return preds, labels

def compute_metrics(pred):
    predictions, labels = pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]
     
    decoded_preds = tokenizer.batch_decode(predictions,
    skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels,
    skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds,
    decoded_labels)

    bleu_output = bleu.compute(predictions=decoded_preds,
    references=decoded_labels)
    return bleu_output

e:\Anaconda\envs\pytorch_test\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
model_name = "t5-base-smallDS-story-generation-1Epoch"
model_dir = f"../Results/{model_name}"


training_args = Seq2SeqTrainingArguments(
                output_dir=model_dir,
                learning_rate=0.001,
                do_eval=True, # will be set to true if evaluation strategy is set
                do_predict=True, #Whether to run predictions on the test set or not.
                num_train_epochs=1,
                evaluation_strategy="epoch", 
                #eval_steps= 100, # Number of update steps between two evaluations if evaluation_strategy="steps". Will default to the same value as logging_steps if not set.
                save_steps=500, # Number of updates steps before two checkpoint saves if save_strategy="steps".
                #max_steps=10, # the total number of training steps to perform
                save_total_limit= 10, # the maximum number of models to keep before deleting the oldest one
                predict_with_generate=True, # Whether to use generate to calculate generative metrics (ROUGE, BLEU).
                generation_num_beams=10,  #The num_beams to use on each evaluation loop when predict_with_generate=True. Will default to the num_beams value of the model configuration
                gradient_checkpointing=True, #
                gradient_accumulation_steps=1, #Number of updates steps to accumulate the gradients for, before performing a backward/update pass
                per_device_train_batch_size=4, #The batch size per GPU/TPU core/CPU for training.
                per_device_eval_batch_size=4, #The batch size per GPU/TPU core/CPU for evaluation.
                optim="adafactor", #The optimizer to use: adamw_hf, adamw_torch, adamw_apex_fused, or adafactor.
                report_to="tensorboard"
)

In [8]:
def model_init():
    """
    Simple function to ensure that a new model is used for finetuning
    """
    return T5ForConditionalGeneration.from_pretrained("t5-base")

trainer = Seq2SeqTrainer(
            model_init=model_init,
            args=training_args,
            train_dataset=small_train,
            eval_dataset=small_val,
            compute_metrics=compute_metrics,
            tokenizer=tokenizer
            )

loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at C:\Users\Simon/.cache\huggingface\transformers\91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_re

In [51]:
# Start TensorBoard before training to monitor it in progress

%load_ext tensorboard
%tensorboard --logdir '{model_dir}'/runs

## Train and Evaluate

In [52]:
trainer.train()

loading configuration file config.json from cache at C:\Users\Simon/.cache\huggingface\hub\models--t5-base\snapshots\23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9\config.json
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "pre

{'loss': 0.2077, 'learning_rate': 0.0006513249651324965, 'epoch': 0.35}


Model weights saved in ../Results/t5-base-smallDS-story-generation-1Epoch\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ../Results/t5-base-smallDS-story-generation-1Epoch\checkpoint-500\tokenizer_config.json
Special tokens file saved in ../Results/t5-base-smallDS-story-generation-1Epoch\checkpoint-500\special_tokens_map.json
Copy vocab file to ../Results/t5-base-smallDS-story-generation-1Epoch\checkpoint-500\spiece.model
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
 70%|██████▉   | 1000/1434 [1:40:03<43:36,  6.03s/it]Saving model checkpoint to ../Results/t5-base-smallDS-story-generation-1Epoch\checkpoint-1000
Configuration saved in ../Results/t5-base-smallDS-story-generation-1Epoch\checkpoint-1000\config.json


{'loss': 0.1419, 'learning_rate': 0.00030264993026499307, 'epoch': 0.7}


Model weights saved in ../Results/t5-base-smallDS-story-generation-1Epoch\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in ../Results/t5-base-smallDS-story-generation-1Epoch\checkpoint-1000\tokenizer_config.json
Special tokens file saved in ../Results/t5-base-smallDS-story-generation-1Epoch\checkpoint-1000\special_tokens_map.json
Copy vocab file to ../Results/t5-base-smallDS-story-generation-1Epoch\checkpoint-1000\spiece.model
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
100%|██████████| 1434/1434 [2:23:36<00:00,  5.27s/it]***** Running Evaluation *****
  Num examples = 687
  Batch size = 4
Trainer is attempting to log a value of "[0.6412949888066127, 0.3923828125, 0.2512959206671174, 0.17243214475784993]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.

100%|██████████| 1434/1434 [2:42:40<00:00,  5.27s/it]

Training

{'eval_loss': 0.1088036298751831, 'eval_bleu': 0.1240426543146586, 'eval_precisions': [0.6412949888066127, 0.3923828125, 0.2512959206671174, 0.17243214475784993], 'eval_brevity_penalty': 0.3838647015118951, 'eval_length_ratio': 0.5108647840239289, 'eval_translation_length': 5807, 'eval_reference_length': 11367, 'eval_runtime': 1143.7467, 'eval_samples_per_second': 0.601, 'eval_steps_per_second': 0.15, 'epoch': 1.0}
{'train_runtime': 9760.5885, 'train_samples_per_second': 0.587, 'train_steps_per_second': 0.147, 'train_loss': 0.15963882914480637, 'epoch': 1.0}


TrainOutput(global_step=1434, training_loss=0.15963882914480637, metrics={'train_runtime': 9760.5885, 'train_samples_per_second': 0.587, 'train_steps_per_second': 0.147, 'train_loss': 0.15963882914480637, 'epoch': 1.0})

In [53]:
trainer.save_model(f"../Models/{model_name}") 

Saving model checkpoint to ../Models/t5-base-smallDS-story-generation-1Epoch
Configuration saved in ../Models/t5-base-smallDS-story-generation-1Epoch\config.json
Model weights saved in ../Models/t5-base-smallDS-story-generation-1Epoch\pytorch_model.bin
tokenizer config file saved in ../Models/t5-base-smallDS-story-generation-1Epoch\tokenizer_config.json
Special tokens file saved in ../Models/t5-base-smallDS-story-generation-1Epoch\special_tokens_map.json
Copy vocab file to ../Models/t5-base-smallDS-story-generation-1Epoch\spiece.model


In [56]:
trainer.evaluate(small_val)
test_op = trainer.predict(small_test)
print(tokenizer.decode(test_op[0][1],skip_special_tokens=True))

***** Running Evaluation *****
  Num examples = 687
  Batch size = 4
100%|██████████| 172/172 [06:02<00:00,  2.11s/it]
***** Running Prediction *****
  Num examples = 1136
  Batch size = 4
100%|██████████| 284/284 [11:03<00:00,  2.34s/it]

A 19-year-old man and a 19-year-old woman were also shot


## Testing

In [57]:
# Comparing input RDF to trainer.predict output
for pred_iteration, predictions in enumerate(test_op[0][:5]):
    print(f"input RDF:  {tokenizer.decode(small_test['input_ids'][pred_iteration],skip_special_tokens=True) }\nGeneration:  {tokenizer.decode(predictions,skip_special_tokens=True)}\n\n")

input RDF:  ['accidentAddress | southeast_Houston', 'accidentDate | overnight','shootingType | violent_shooting']
Generation:  A violent shooting occurred overnight in southeast Houston.


input RDF:  ['victimAge | 19', 'victimGender | men']
Generation:  A 19-year-old man and a 19-year-old woman were also shot


input RDF:  ['victimAge | 39-year-old', 'victimGender | female', 'victimRace | black', 'victimStatus | major_injuries']
Generation:  A 39-year-old woman and a 39-year-old woman with major injuries


input RDF:  ['victimName | Adrian_Potts']
Generation:  Adrian Potts was pronounced dead at the scene.


input RDF:  ["hospitalName | Cook_Children's_Medical_Center", 'takenToHospital | True', 'victimAge | 3-year-olds', 'victimAge_Group | children', 'victimNumber | two', 'victimStatus | serious_condition']
Generation:  The children were taken to Cook Children's Medical Center in serious condition.




In [2]:
# get the saved model and tokenizer
saved_model = T5ForConditionalGeneration.from_pretrained('../Models/t5-base-smallDS-story-generation-1Epoch/', local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained('../Models/t5-base-smallDS-story-generation-1Epoch/',local_files_only=True)

In [8]:
#split the testset into input and output, so that we easily generate predictions and compare them to the true version
true_articles = small_test['labels']
input_information = small_test.remove_columns("labels")

# set-up a dataloader to load in the tokenized test dataset
test_dataloader = torch.utils.data.DataLoader(input_information, batch_size=32)

# generate text for each batch
all_predictions = []
for i,batch in enumerate(test_dataloader):
  #if i < 5:
  predictions = saved_model.generate(**batch) #, min_length=30, max_length=100, num_beams=5, top_p=0.6, repetition_penalty=1.3)
  #print(f'prediction: {tokenizer.decode(predictions[0],skip_special_tokens=True)}  ') # 
  all_predictions.append(predictions)
  #else:
  #  break

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]



e:\Anaconda\envs\pytorch_test\lib\site-packages\transformers\generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [58]:
# Comparing input RDF to trainer.predict output
for pred_iteration, predictions in enumerate(true_articles[0][:5]):
    print(f"Gold:  {tokenizer.decode(true_articles[pred_iteration],skip_special_tokens=True) }\nGeneration:  {tokenizer.decode(all_predictions_flattened[pred_iteration],skip_special_tokens=True)}\n\n")

Gold:  A bar fight ends in a violent shooting overnight in southeast Houston.
Generation:  Police are investigating a violent shooting in southeast Houston overnight.


Gold:  The wounded men range in age from 19 to 21.
Generation:  The men, who are all expected to survive, are between 19 and 19 years old.


Gold:  39-year-old black female with major injuries
Generation:  A 39-year-old female was listed in critical condition and a male was listed in


Gold:  Police on Saturday night identified him as Adrian Potts.
Generation:  Adrian Potts was shot in the right leg, but he was never hurt.


Gold:  The two children, both 3-year-olds, were transported in serious condition to Cook Children's Medical Center.
Generation:  The children, who are all taken to Cook Chili's Medical Center in serious condition, are




#### pickle generated predictions and gold standard 

In [55]:
import pickle
with open(f'Predictions_{model_name}.pickle', 'wb') as f:
    pickle.dump(all_predictions_flattened, f)

with open(f'Labels_{model_name}.pickle', 'wb') as f:
    pickle.dump(true_articles, f)

e:\Anaconda\envs\pytorch_test\lib\site-packages\torch\storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [ ]:
# compute metrics
predictions_labels = [all_predictions_flattened, true_articles]
compute_metrics(predictions_labels) # This calculates blue metric

In [9]:
decoded_predictions = []
decoded_true_articles = []

for iteration, text in enumerate(true_articles):
    decoded_predictions.append(tokenizer.decode(all_predictions_flattened[iteration],skip_special_tokens=True))
    decoded_true_articles.append(tokenizer.decode(text,skip_special_tokens=True))

#### Rouge test

In [21]:
import evaluate
rouge = evaluate.load('rouge')
rouge_results = rouge.compute(predictions=decoded_predictions, references=decoded_true_articles)
rouge_results

{'rouge1': 0.49186400017973353,
 'rouge2': 0.29073247525987433,
 'rougeL': 0.4282798132067188,
 'rougeLsum': 0.4283660260971445}

#### Bertscore

In [102]:
from evaluate import load
bertscore = load("bertscore")
#predictions = ["hello there", "general kenobi"]
##references = ["hello there", "general kenobi"]
bertscore_results = bertscore.compute(predictions=decoded_predictions, references=decoded_true_articles, lang="en")

In [103]:
print(bertscore_results['f1'])

[0.9499719738960266, 0.8786465525627136, 0.9425523281097412, 0.888322651386261, 0.9639273881912231, 0.9058011770248413, 0.922014057636261, 0.9419782757759094, 0.872467041015625, 0.9157440662384033, 0.9387620091438293, 0.8557538390159607, 0.9083451628684998, 0.9227930307388306, 0.9603692889213562, 0.9999998807907104, 0.9119024276733398, 0.9661831259727478, 0.9350420832633972, 0.8840372562408447, 0.9284369945526123, 0.8676361441612244, 0.8574816584587097, 0.9095780849456787, 0.9454705119132996, 0.9572874307632446, 0.9291267395019531, 0.9404165744781494, 0.8495192527770996, 0.8651273250579834, 0.8772913217544556, 0.9339597225189209, 0.9348630309104919, 0.8939517736434937, 0.9057127237319946, 0.9151082038879395, 0.9366471767425537, 0.90410315990448, 0.9384956955909729, 0.9110017418861389, 0.8989789485931396, 0.9116221070289612, 0.9441954493522644, 0.972309410572052, 0.9292705655097961, 0.8936586976051331, 0.9400068521499634, 0.8933810591697693, 0.8447283506393433, 0.9088520407676697, 0.914

#### Perplexity

In [109]:
from evaluate import load
perplexity = load("perplexity", module_type="metric")
perp_results = perplexity.compute(predictions=decoded_predictions, model_id='gpt2')
perp_results["mean_perplexity"]

Using pad_token, but it is not set yet.
100%|██████████| 71/71 [00:10<00:00,  6.57it/s]


761.7258675585331

#### Meteor

In [110]:
meteor = evaluate.load('meteor')
meteor_results = meteor.compute(predictions=decoded_predictions, references=decoded_true_articles)
meteor_results

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Simon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Simon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Simon\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.36306766407957697}

### BARTScore

https://github.com/neulab/BARTScore

In [12]:
### Code for Bartscore
# %%
import torch
import torch.nn as nn
import traceback
from transformers import BartTokenizer, BartForConditionalGeneration


class BARTScorer:
    def __init__(self, device='cuda:0', max_length=256, checkpoint='facebook/bart-large-cnn'):
        # Set up model
        self.device = device
        self.max_length = max_length
        self.tokenizer = BartTokenizer.from_pretrained(checkpoint)
        self.model = BartForConditionalGeneration.from_pretrained(checkpoint)
        self.model.eval()
        self.model.to(device)

        # Set up loss
        self.loss_fct = nn.NLLLoss(reduction='none', ignore_index=self.model.config.pad_token_id)
        self.lsm = nn.LogSoftmax(dim=1)

    def load(self, path=None):
        """ Load model from paraphrase finetuning """
        if path is None:
            path = 'models/bart.pth'
        self.model.load_state_dict(torch.load(path, map_location=self.device))

    def score(self, srcs, tgts, batch_size=4):
        """ Score a batch of examples """
        score_list = []
        for i in range(0, len(srcs), batch_size):
            src_list = srcs[i: i + batch_size]
            tgt_list = tgts[i: i + batch_size]
            try:
                with torch.no_grad():
                    encoded_src = self.tokenizer(
                        src_list,
                        max_length=self.max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    encoded_tgt = self.tokenizer(
                        tgt_list,
                        max_length=self.max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    src_tokens = encoded_src['input_ids'].to(self.device)
                    src_mask = encoded_src['attention_mask'].to(self.device)

                    tgt_tokens = encoded_tgt['input_ids'].to(self.device)
                    tgt_mask = encoded_tgt['attention_mask']
                    tgt_len = tgt_mask.sum(dim=1).to(self.device)

                    output = self.model(
                        input_ids=src_tokens,
                        attention_mask=src_mask,
                        labels=tgt_tokens
                    )
                    logits = output.logits.view(-1, self.model.config.vocab_size)
                    loss = self.loss_fct(self.lsm(logits), tgt_tokens.view(-1))
                    loss = loss.view(tgt_tokens.shape[0], -1)
                    loss = loss.sum(dim=1) / tgt_len
                    curr_score_list = [-x.item() for x in loss]
                    score_list += curr_score_list

            except RuntimeError:
                traceback.print_exc()
                print(f'source: {src_list}')
                print(f'target: {tgt_list}')
                exit(0)
        return score_list

    def multi_ref_score(self, srcs, tgts, agg="mean", batch_size=4):
        # Assert we have the same number of references
        ref_nums = [len(x) for x in tgts]
        if len(set(ref_nums)) > 1:
            raise Exception("You have different number of references per test sample.")

        ref_num = len(tgts[0])
        score_matrix = []
        for i in range(ref_num):
            curr_tgts = [x[i] for x in tgts]
            scores = self.score(srcs, curr_tgts, batch_size)
            score_matrix.append(scores)
        if agg == "mean":
            score_list = np.mean(score_matrix, axis=0)
        elif agg == "max":
            score_list = np.max(score_matrix, axis=0)
        else:
            raise NotImplementedError
        return list(score_list)

    def test(self, batch_size=3):
        """ Test """
        src_list = [
            'This is a very good idea. Although simple, but very insightful.',
            'Can I take a look?',
            'Do not trust him, he is a liar.'
        ]

        tgt_list = [
            "That's stupid.",
            "What's the problem?",
            'He is trustworthy.'
        ]

        print(self.score(src_list, tgt_list, batch_size))

In [13]:
bart_scorer = BARTScorer(device='cuda:0', checkpoint='facebook/bart-large-cnn')

In [15]:
bart_scores = bart_scorer.score(srcs=decoded_predictions, tgts=decoded_true_articles, batch_size=1) 

In [17]:
print(len(bart_scores))
bart_scores

1136


[-2.5916852951049805,
 -2.8240702152252197,
 -1.751649260520935,
 -3.2559080123901367,
 -1.5100139379501343,
 -4.154699802398682,
 -2.406635284423828,
 -2.4597301483154297,
 -3.315011739730835,
 -3.2063775062561035,
 -1.6033596992492676,
 -3.407088279724121,
 -2.2348411083221436,
 -2.855161428451538,
 -1.1250760555267334,
 -0.4073900282382965,
 -2.483037233352661,
 -1.2667436599731445,
 -2.0463755130767822,
 -3.5058274269104004,
 -1.4195690155029297,
 -3.1969618797302246,
 -2.915322780609131,
 -2.538642644882202,
 -1.7162305116653442,
 -1.9518054723739624,
 -1.8608429431915283,
 -1.7569050788879395,
 -4.102962970733643,
 -4.241076946258545,
 -3.9985783100128174,
 -1.7357791662216187,
 -1.4153032302856445,
 -2.3941750526428223,
 -2.353602170944214,
 -2.5307979583740234,
 -2.136995315551758,
 -2.352670907974243,
 -2.213817834854126,
 -1.5588377714157104,
 -1.8158177137374878,
 -2.2451202869415283,
 -1.289177417755127,
 -1.0434327125549316,
 -2.2734375,
 -1.9201128482818604,
 -1.434446930

#### BLEURT ---> Giving TF error?

https://github.com/google-research/bleurt


Tf error occurs, this is a known issue. Apparently this PR should fix it: https://github.com/google-research/bleurt/pull/44 --> Fix doesnt seem so work

In [51]:
#code for Bleurt
#from bleurt import score
import bleurt.score as score


In [6]:
import pickle
# open a file, where you stored the pickled data
file = open('../Testingdata/Labels_t5-base-smallDS-story-generation-1Epoch.pickle', 'rb')
# dump information to that file
all_predictions_flattened = pickle.load(file)
# close the file
file.close()

# open a file, where you stored the pickled data
file = open('../Testingdata/Predictions_t5-base-smallDS-story-generation-1Epoch.pickle', 'rb')
# dump information to that file
true_articles = pickle.load(file)
# close the file
file.close()

In [7]:
decoded_predictions = []
decoded_true_articles = []

for iteration, text in enumerate(true_articles):
    decoded_predictions.append(tokenizer.decode(all_predictions_flattened[iteration],skip_special_tokens=True))
    decoded_true_articles.append(tokenizer.decode(text,skip_special_tokens=True))

In [49]:
checkpoint = "bleurt/test_checkpoint"
references = decoded_true_articles
candidates = decoded_predictions

In [52]:
scorer = score.BleurtScorer(checkpoint)
scores = scorer.score(references=references, candidates=candidates)
assert type(scores) == list and len(scores) == 1
print(scores)

INFO:tensorflow:Reading checkpoint bleurt/test_checkpoint.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint dbleurt_tiny
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:dbleurt_tiny
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.


InvalidArgumentError: cannot compute __inference_pruned_4209 as input #0(zero-based) was expected to be a int64 tensor but is a int32 tensor [Op:__inference_pruned_4209]

### Visualizations (SHAP / ECCO)

In [59]:
import ecco

lm = ecco.from_pretrained('../Models/t5-base') #t5-base-smallDS-story-generation-1Epoch

ValueError: ("The model '../Models/t5-base' is not defined in Ecco's 'model-config.yaml' file and so is not explicitly supported yet. Supported models are:", ['gpt2', 'gpt2-medium', 'gpt2-xl', 'distilgpt2', 'distilbert-base-uncased', 'bert-base-uncased', 'bert-large-uncased', 'distilbert-base-uncased-finetuned-sst-2-english', 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract', 'albert-base-v2', 'albert-large-v2', 'albert-xxlarge-v2', 'distilroberta-base', 'roberta-base', 'roberta-large', 'google/electra-small-discriminator', 'google/electra-base-discriminator', 'sentence-transformers/distilbert-base-nli-stsb-mean-tokens', 't5-small', 't5-base', 'bigscience/T0_3B', 'bigscience/T0', 'bigscience/T0p', 'bigscience/T0pp', 'tscholak/1wnr382e', 'valhalla/t5-small-qg-hl', 'valhalla/t5-small-qa-qg-hl', 'valhalla/t5-base-e2e-qg', 'facebook/bart-large-mnli', 'sshleifer/tiny-gpt2', 'julien-c/bert-xsmall-dummy', 'EleutherAI/gpt-neo-125M', 'EleutherAI/gpt-neo-1.3B', 'EleutherAI/gpt-neo-2.7B'])

In [ ]:
## SHAP
explainer = shap.DeepExplainer(model, background)
shap_values = explainer.shap_values(x_test_each_class)


In [8]:
import shap
#https://shap.readthedocs.io/en/latest/text_examples.html

In [10]:
# load a transformers pipeline model
#model = transformers.pipeline('sentiment-analysis', return_all_scores=True)
#model = T5ForConditionalGeneration.from_pretrained('../Models/t5-base-smallDS-story-generation-1Epoch/', local_files_only=True)

model_pipeline = pipeline('text-generation', model=saved_model, tokenizer=tokenizer)




The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['XGLMForCausalLM', 'PLBartForCausalLM', 'QDQBertLMHeadModel', 'TrOCRForCausalLM', 'GPTJForCausalLM', 'RemBertForCausalLM', 'RoFormerForCausalLM', 'BigBirdPegasusForCausalLM', 'GPTNeoForCausalLM', 'BigBirdForCausalLM', 'CamembertForCausalLM', 'XLMRobertaXLForCausalLM', 'XLMRobertaForCausalLM', 'RobertaForCausalLM', 'BertLMHeadModel', 'OpenAIGPTLMHeadModel', 'GPT2LMHeadModel', 'TransfoXLLMHeadModel', 'XLNetLMHeadModel', 'XLMWithLMHeadModel', 'ElectraForCausalLM', 'CTRLLMHeadModel', 'ReformerModelWithLMHead', 'BertGenerationDecoder', 'XLMProphetNetForCausalLM', 'ProphetNetForCausalLM', 'BartForCausalLM', 'OPTForCausalLM', 'MBartForCausalLM', 'PegasusForCausalLM', 'MarianForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'MegatronBertForCausalLM', 'Speech2Text2ForCausalLM', 'Data2VecTextForCausalLM'].


In [21]:

# explain the model on two sample inputs
#explainer = shap.Explainer(model_pipeline) 
explainer = shap.Explainer(saved_model, tokenizer)
shap_values = explainer(["What a great movie!"])

# visualize the first prediction's explanation for the POSITIVE output class
#shap.plots.text(shap_values[0, :, "POSITIVE"])


In [22]:
shap.plots.text(shap_values)

In [23]:
shap_values = explainer(['Scientists confirmed the worst possible outcome: the massive asteroid will collide with Earth'])

Partition explainer: 2it [00:17, 17.29s/it]               


In [24]:
shap.plots.text(shap_values)
